In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path
import h5py
import numpy as np
import torch
import os, os.path as osp
from tqdm import tqdm
import matplotlib.pyplot as plt

from datasets.scannet.utils_3d import ProjectionHelper, adjust_intrinsic, make_intrinsic, load_intrinsic, load_pose
from datasets.scannet.utils_3d import load_depth, load_color

In [ ]:
def get_scan_name(scene_id, scan_id):
    return f'scene{str(scene_id).zfill(4)}_{str(scan_id).zfill(2)}'

# globals
subvol_size = (32, 32, 64)
voxel_size = 0.05
voxel_dims = (1, 1, 1)
root = Path('/mnt/data/scannet/scans')
proj_img_size = (40, 30)

data_dir = Path('/mnt/data/scannet/backproj')
fname = 'val-v7.h5'
f = h5py.File(data_dir / fname, 'r')
f['x'].shape

In [ ]:
def stats(x, msg=''):
    print(f'{msg} min: {int(x.min())}, max: {int(x.max())}, avg: {int(x.mean())}')

In [ ]:
occupied, projected, overlap = [], [], []
skipped = 0
for ndx in tqdm(range(f['x'].shape[0])):
    w2g, sceneid, scanid, frames = f['world_to_grid'][ndx], f['scene_id'][ndx], f['scan_id'][ndx], f['frames'][ndx]

    subvol_x = f['x'][ndx]
    occupied.append((subvol_x == 1).sum())
    # per-scene basics
    scan_name = get_scan_name(sceneid, scanid)
    frame_ndx = 0
    # val set - no frame, skip
    if frames[frame_ndx] == -1:
        skipped += 1
        continue
    pose_path = root / scan_name / 'pose' / f'{frames[frame_ndx]}.txt'
    pose = load_pose(pose_path).numpy()
    depth_path = root / scan_name / 'depth' / f'{frames[frame_ndx]}.png' 
    depth = load_depth(depth_path, proj_img_size)
    # get projection
    intrinsic_path = root / scan_name / 'intrinsic/intrinsic_color.txt'
    intrinsic = load_intrinsic(intrinsic_path)
    # adjust for smaller image size
    intrinsic = adjust_intrinsic(intrinsic, [1296, 968], proj_img_size)

    projection = ProjectionHelper(
                intrinsic, 
                0.4, 4.0,
                proj_img_size,
                subvol_size, voxel_size
            )

    # projection expects origin of chunk in a corner
    # but w2g is wrt center of the chunk -> add 16 to its "grid coords" to get the required grid coords
    # ie 0,0,0 becomes 16,16,16
    # add an additional translation to existing one 
    t = torch.eye(4)
    t[:3, -1] = torch.Tensor(subvol_size)/2
    w2g_tmp = t @ w2g

    proj = projection.compute_projection(torch.Tensor(depth), torch.Tensor(pose), torch.Tensor(w2g_tmp))
    if proj is None: 
        continue
    proj3d, proj2d = proj
    num_inds = proj3d[0]

    ind3d = proj3d[1:1+num_inds]
    ind2d = proj2d[1:1+num_inds]

    coords_3d = ProjectionHelper.lin_ind_to_coords_static(ind3d, subvol_size).T[:, :-1].long()
    i,j,k = coords_3d.T

    projected.append(proj3d[0].item())
    
    overlap.append((subvol_x[i, j, k] == 1).sum())

print('Skipped:', skipped)
projected = torch.Tensor(projected)
occupied = torch.Tensor(occupied)
overlap = torch.Tensor(overlap)

In [ ]:
stats(occupied, 'occupied')
stats(projected, 'projected')
stats(overlap, 'overlap')

# Indices vs coords - 3D

In [ ]:
# set these many features in the volume
num_ind = 10
# pick the inds to set features
inds = torch.randperm(32*32*32)[:num_ind]
coords_3d = ProjectionHelper.lin_ind_to_coords_static(inds, subvol_size).T[:, :-1].long()
i,j,k = coords_3d.T

In [ ]:
# %%timeit -n1000 pass

# empty features tensor CWHD
x = torch.zeros(2, 32, 32, 32)
# set using indices -> dont reshape, may create a new tensor
# CDHW
x = x.permute(0, 3, 2, 1).contiguous()
x.view(2, -1)[:, inds] = torch.ones(2, num_ind)
# back to CWHD
x = x.permute(0, 3, 2, 1)

In [ ]:
# %%timeit -n1000 pass

x = torch.zeros(2, 32, 32, 32)
coords_3d = ProjectionHelper.lin_ind_to_coords_static(inds, subvol_size).T[:, :-1].long()
i,j,k = coords_3d.T

# empty features tensor CWHD
x = torch.zeros(2, 32, 32, 32)

# set values with ijk
x[:, i, j, k] = torch.ones(2, num_ind)

In [ ]:
# CWHD, then use coords
print(x[:, i, j, k])

In [ ]:
# CDHW, then use inds
print(x.permute(0, 3, 2, 1).reshape(2, -1)[:, inds])

# Indices vs coords in 2D

In [ ]:
img_size = (40, 30)
num_ind = 10
inds2d = torch.randperm(img_size[0]*img_size[1])[:num_ind]
# coords
coords_2d = ProjectionHelper.lin_ind2d_to_coords2d_static(inds2d, (img_size))
i, j = coords_2d

In [ ]:
%%timeit -n1000 pass

# CHW image
x2d = torch.zeros(2, img_size[1], img_size[0])

# set features using linear indices in CHW
# CWH
x2d.view(2, -1)[:, inds2d] = torch.ones(2, num_ind)

In [ ]:
%%timeit -n1000 pass

# CHW image
x2d = torch.zeros(2, img_size[1], img_size[0])

# get coords
coords_2d = ProjectionHelper.lin_ind2d_to_coords2d_static(inds2d, (img_size))
i, j = coords_2d

# set features in CWH using coords
x2d = x2d.permute(0, 2, 1)
x2d[:, i, j] = torch.ones(2, num_ind)
# back to CHW
x2d = x2d.permute(0, 2, 1)


In [ ]:
# use coords to index into CWH
x2d.permute(0, 2, 1)[:, i, j]

In [ ]:
# use inds to index into CHW
x2d.view(2, -1)[:, inds2d]